In [1]:
# Import packages
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn_pandas import DataFrameMapper

from sklearn import metrics
from sklearn.metrics import classification_report,confusion_matrix, ConfusionMatrixDisplay, accuracy_score

C:\Users\acer\AppData\Roaming\Python\Python37\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


# Reading the Data

In [2]:
train_data = pd.read_csv("../../Data/Final datasets/train_data.csv")
val_data = pd.read_csv("../../Data/Final datasets/val_data.csv")
test_data = pd.read_csv("../../Data/Final datasets/test_data.csv")

train_data.sample(5)

,Unnamed: 0,char_count,word_count,sentence_count,prop_unique_words,avg_sentence_length,prop_punctuations,prop_stopwords,prop_words_in_quotes,prop_nouns,prop_verbs,prop_adjectives,prop_discourse_relations,textblob_sentiment,text,class_label,text_preprocessed
20912,20912,0.229954,0.181304,0.000000,0.583429,0.515881,0.332684,1.000000,1.0,0.649008,1.000000,0.675265,0.000000,0.500000,The Arrivals Bosanski Prijevod 36-theStoryOfJe...,1,arriv bosanski prijevod 36thestoryofjesussuppo...
11644,11644,0.539677,0.541733,0.356333,0.469383,0.632330,0.362808,0.833539,1.0,0.501173,0.535221,0.518292,0.322086,0.502741,Half of Germans want new elections after coali...,0,half german want new elect coalit talk fail be...
15508,15508,0.566192,0.566374,0.422957,0.512227,0.597302,0.317863,0.826211,1.0,0.530938,0.658307,0.474156,0.204380,0.503698,Paul Ryan: Failure of Obamacare Replacement Ma...,0,paul ryan failur obamacar replac make tax refo...
12020,12020,0.551511,0.553166,0.401542,0.510845,0.601535,0.344543,0.820188,1.0,0.505343,0.570749,0.579872,0.289256,0.491288,Having nuclear weapons 'matter of life and dea...,0,nuclear weapon matter life death north korea a...
28157,28157,0.530959,0.524410,0.429420,0.636065,0.534970,0.391446,0.836109,1.0,0.520611,0.639611,0.556508,0.227437,0.536869,Republican presidential candidate Cruz propose...,0,republican presidenti candid cruz propos milit...


In [3]:
X_train_text = train_data["text_preprocessed"].values
y_train = train_data["class_label"].values

X_val_text = val_data["text_preprocessed"].values
y_val = val_data["class_label"].values

X_test_text = test_data["text_preprocessed"].values
y_test = test_data["class_label"].values

# Baseline Model (Random Forest)
## Using CountVectoriser with Bag of Words, Unigrams + Bigrams

### Using 134 and 3k features

In [4]:
# with 134 features
vectorizer = CountVectorizer(stop_words='english', ngram_range=(1,2), min_df=0.15)
vectorizer.fit(X_train_text)

X_train = vectorizer.transform(X_train_text)
X_val = vectorizer.transform(X_val_text)
X_test = vectorizer.transform(X_test_text)

In [5]:
print("number of features used:", len(vectorizer.get_feature_names()))

number of features used: 134


In [9]:
# initialise Decision Tree
clf = RandomForestClassifier(n_estimators=100, random_state=0) 

# train model
clf.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

In [ ]:
y_val_pred = clf.predict(X_val)
print(classification_report(y_val, y_val_pred))

In [ ]:
y_test_pred = clf.predict(X_test)
print(classification_report(y_test, y_test_pred))

In [ ]:
#  with 3k features
vectorizer = CountVectorizer(stop_words='english', ngram_range=(1,2), min_df=0.01)
vectorizer.fit(X_train_text)

X_train = vectorizer.transform(X_train_text)
X_val = vectorizer.transform(X_val_text)
X_test = vectorizer.transform(X_test_text)

In [ ]:
# train model
clf.fit(X_train, y_train)

In [ ]:
y_val_pred = clf.predict(X_val)
print(classification_report(y_val, y_val_pred))

In [ ]:
y_test_pred = clf.predict(X_test)
print(classification_report(y_test, y_test_pred))

### Using 134 and 3k with all added features

In [13]:
# with 134 features
vectorizer = CountVectorizer(stop_words='english', ngram_range=(1,2), min_df=0.15)
vectorizer.fit(X_train_text)

X_train = vectorizer.transform(X_train_text)
X_val = vectorizer.transform(X_val_text)
X_test = vectorizer.transform(X_test_text)

## Using Tf-ldf
### Feature Selection

In [10]:
# with 134 features
tfidf_params = {'unigram and bigram': (1,2)}

for ngram, values in tfidf_params.items():
    tfidf_vectorizer = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=values, min_df=0.15)
    tfidf_vectorizer.fit(X_train_text)

    X_train = tfidf_vectorizer.transform(X_train_text)
    X_val = tfidf_vectorizer.transform(X_val_text)
    X_test = tfidf_vectorizer.transform(X_test_text)

    print(f"Model with {ngram}")
    clf.fit(X_train, y_train)

    # Validation Data
    print("Testing using validation data:")    
    y_val_pred = clf.predict(X_val)
    print(classification_report(y_val, y_val_pred))
    print("------------------------------------------")

    # Test Data
    print("Testing using test data:")  
    y_test_pred = clf.predict(X_test)
    print(classification_report(y_test, y_test_pred))
    print("------------------------------------------")
    print("------------------------------------------")

Model with unigram and bigram
Testing using validation data:
              precision    recall  f1-score   support

           0       0.96      0.90      0.93      6361
           1       0.91      0.96      0.94      6659

    accuracy                           0.93     13020
   macro avg       0.93      0.93      0.93     13020
weighted avg       0.93      0.93      0.93     13020

------------------------------------------
Testing using test data:
              precision    recall  f1-score   support

           0       0.96      0.91      0.94      6361
           1       0.92      0.96      0.94      6660

    accuracy                           0.94     13021
   macro avg       0.94      0.94      0.94     13021
weighted avg       0.94      0.94      0.94     13021

------------------------------------------
------------------------------------------


In [11]:
tfidf_params = {'unigram and bigram': (1,2)}

for ngram, values in tfidf_params.items():
    tfidf_vectorizer = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=values, min_df=0.01)
    tfidf_vectorizer.fit(X_train_text)

    X_train = tfidf_vectorizer.transform(X_train_text)
    X_val = tfidf_vectorizer.transform(X_val_text)
    X_test = tfidf_vectorizer.transform(X_test_text)

    print(f"Model with {ngram}")
    clf.fit(X_train, y_train)

    # Validation Data
    print("Testing using validation data:")    
    y_val_pred = clf.predict(X_val)
    print(classification_report(y_val, y_val_pred))
    print("------------------------------------------")

    # Test Data
    print("Testing using test data:")
    y_test_pred = clf.predict(X_test)
    print(classification_report(y_test, y_test_pred))

Model with unigram and bigram
Testing using validation data:
              precision    recall  f1-score   support

           0       0.97      0.94      0.96      6361
           1       0.95      0.97      0.96      6659

    accuracy                           0.96     13020
   macro avg       0.96      0.96      0.96     13020
weighted avg       0.96      0.96      0.96     13020

------------------------------------------
Testing using test data:
              precision    recall  f1-score   support

           0       0.97      0.95      0.96      6361
           1       0.95      0.97      0.96      6660

    accuracy                           0.96     13021
   macro avg       0.96      0.96      0.96     13021
weighted avg       0.96      0.96      0.96     13021



### Feature selection with all added features

In [19]:
test_data_added_features_scaled = pd.read_csv("../../Data/Final datasets/train_data.csv")
# val_data_added_features_scaled = pd.read_csv("../../Data/Final datasets/val_data.csv")
test_data_added_features_scaled = pd.read_csv("../../Data/Final datasets/test_data.csv")

In [20]:
y_train = train_data_added_features_scaled["class_label"].values
y_test = test_data_added_features_scaled["class_label"].values
# y_val = val_data_features["class_label"].values

In [21]:
# with 134 features
tfidf_vectorizer = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1,2), min_df=0.15)

# Create mapper object to combine added features and tfidf word vectors
mapper = DataFrameMapper([
    (['char_count', 'word_count', 'sentence_count', 'prop_unique_words',
    'avg_sentence_length', 'prop_punctuations', 'prop_stopwords',
    'prop_words_in_quotes', 'prop_nouns', 'prop_verbs', 'prop_adjectives',
    'prop_discourse_relations', 'textblob_sentiment'], None), 
    ('text_preprocessed', tfidf_vectorizer)
])

X_train_added_features = mapper.fit_transform(train_data_added_features_scaled)
X_test_added_features = mapper.transform(test_data_added_features_scaled)

clf.fit(X_train_added_features, y_train)

# Test Data
print("Testing using test data:")
y_test_pred = clf.predict(X_test_added_features)
print(classification_report(y_test, y_test_pred))

Testing using test data:
              precision    recall  f1-score   support

           0       0.96      0.90      0.93      6361
           1       0.91      0.97      0.94      6660

    accuracy                           0.94     13021
   macro avg       0.94      0.93      0.94     13021
weighted avg       0.94      0.94      0.94     13021



In [22]:
# with 3k features
tfidf_vectorizer = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1,2), min_df=0.01)

# Create mapper object to combine added features and tfidf word vectors
mapper = DataFrameMapper([
    (['char_count', 'word_count', 'sentence_count', 'prop_unique_words',
    'avg_sentence_length', 'prop_punctuations', 'prop_stopwords',
    'prop_words_in_quotes', 'prop_nouns', 'prop_verbs', 'prop_adjectives',
    'prop_discourse_relations', 'textblob_sentiment'], None), 
    ('text_preprocessed', tfidf_vectorizer)
])

X_train_added_features = mapper.fit_transform(train_data_added_features_scaled)
X_test_added_features = mapper.transform(test_data_added_features_scaled)

clf.fit(X_train_added_features, y_train)

# Test Data
print("Testing using test data:")
y_test_pred = clf.predict(X_test_added_features)
print(classification_report(y_test, y_test_pred))

Testing using test data:
              precision    recall  f1-score   support

           0       0.98      0.94      0.96      6361
           1       0.95      0.98      0.96      6660

    accuracy                           0.96     13021
   macro avg       0.96      0.96      0.96     13021
weighted avg       0.96      0.96      0.96     13021



### Feature selection with selected added features

In [23]:
# with 134 features
tfidf_vectorizer = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1,2), min_df=0.15)

# Create mapper object to combine added features and tfidf word vectors
mapper = DataFrameMapper([
    (['char_count', 'word_count', 'prop_unique_words', 'avg_sentence_length', 'prop_punctuations', 'prop_stopwords', 'prop_nouns'], None), 
    ('text_preprocessed', tfidf_vectorizer)
])

X_train_added_features = mapper.fit_transform(train_data_added_features_scaled)
X_test_added_features = mapper.transform(test_data_added_features_scaled)

clf.fit(X_train_added_features, y_train)

# Test Data
print("Testing using test data:")
y_test_pred = clf.predict(X_test_added_features)
print(classification_report(y_test, y_test_pred))

Testing using test data:
              precision    recall  f1-score   support

           0       0.96      0.90      0.93      6361
           1       0.91      0.97      0.94      6660

    accuracy                           0.94     13021
   macro avg       0.94      0.93      0.93     13021
weighted avg       0.94      0.94      0.93     13021



In [24]:
# with 3k features
tfidf_vectorizer = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1,2), min_df=0.01)

# Create mapper object to combine added features and tfidf word vectors
mapper = DataFrameMapper([
    (['char_count', 'word_count', 'prop_unique_words', 'avg_sentence_length', 'prop_punctuations', 'prop_stopwords', 'prop_nouns'], None), 
    ('text_preprocessed', tfidf_vectorizer)
])

X_train_added_features = mapper.fit_transform(train_data_added_features_scaled)
X_test_added_features = mapper.transform(test_data_added_features_scaled)

clf.fit(X_train_added_features, y_train)

# Test Data
print("Testing using test data:")
y_test_pred = clf.predict(X_test_added_features)
print(classification_report(y_test, y_test_pred))

Testing using test data:
              precision    recall  f1-score   support

           0       0.97      0.94      0.96      6361
           1       0.95      0.98      0.96      6660

    accuracy                           0.96     13021
   macro avg       0.96      0.96      0.96     13021
weighted avg       0.96      0.96      0.96     13021

